In [28]:
!pip install names

    100% |████████████████████████████████| 798kB 12.8MB/s ta 0:00:01
  Stored in directory: /Users/flatironschool/Library/Caches/pip/wheels/f9/a5/e1/be3e0aaa6fa285575078fa2aafd9959b45bdbc8de8a6803aeb
Successfully built names


In [74]:
import nltk
from nltk.corpus import stopwords
import string
import names
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
import pandas as pd
import pickle
import numpy as np

# Import Data and Split into Training and Test Sets

In [2]:
horror_scripts_raw = pickle.load(open('hor_scripts.p','rb'))

In [3]:
romance_scripts_raw = pickle.load(open('rom_scripts.p','rb'))

In [4]:
fantasy_scripts_raw = pickle.load(open('fan_scripts.p','rb'))

In [5]:
def dict_shuffle(dict_):
    l = list(dict_.items())
    random.shuffle(l)
    dict_1 = dict(l)
    return dict_1

In [6]:
horror_scripts_raw_shuff = dict_shuffle(horror_scripts_raw)

In [7]:
romance_scripts_raw_shuff = dict_shuffle(romance_scripts_raw)

In [8]:
fantasy_scripts_raw_shuff = dict_shuffle(fantasy_scripts_raw)

In [ ]:
# label = {'Romance':0,'Horror':2,'Fantasy':3}

In [9]:
len(horror_scripts_raw_shuff.keys()), len(romance_scripts_raw_shuff.keys()), len(fantasy_scripts_raw_shuff.keys())

(147, 197, 116)

In [10]:
len(horror_scripts_raw_shuff['28 DAYS LATER'])

110566

In [11]:
keys_list_train = []
keys_list_test = []
X_train = []
X_test = []
y_train = []
y_test = []

In [12]:
def train_test_split(data, feat_train, label_train, feat_test, label_test, label):
    counter = 0
    for k,v in data.items():
        if counter < 80:
            print('train',k,counter);
            feat_train.append(v)
            keys_list_train.append(k)
            label_train.append(label)
            counter +=1
        elif counter >= 80 and counter < 100:
            print('test',k,counter);
            feat_test.append(v)
            keys_list_test.append(k)
            label_test.append(label)
            counter +=1

In [13]:
train_test_split(romance_scripts_raw_shuff,X_train,y_train,X_test,y_test,0)
train_test_split(horror_scripts_raw_shuff,X_train,y_train,X_test,y_test,1)
train_test_split(fantasy_scripts_raw_shuff,X_train,y_train,X_test,y_test,2)

train THE BACK-UP PLAN 0
train PRIDE AND PREJUDICE 1
train "BONNIE & CLYDE" 2
train ST. ELMO'S FIRE 3
train T H E   A M E R I C A N   P R E S I D E N T 4
train LOG #028 5
train MONTE CARLO 6
train PUNCH-DRUNK LOVE by Paul Thomas Anderson 7
train "CRASH" 8
train MRS BROWN BY JEREMY BROCK 9
train THE BROTHERS BLOOM 10
train NINE 11
train R U N A W A Y   B R I D E 12
train UP IN THE AIR 13
train SHREK 14
train DECEPTION 15
train <b><!-- 16
train AS GOOD AS IT GETSAS GOOD AS IT GETS 17
train BLAST FROM THE PAST - by Bill Kelly and Hugh Wilson 18
train HER 19
train EASY "A" 20
train INT. PUBLISHING HOUSE RECEPTION AREA - DAY 21
train WILLIAM SHAKESPEARE'S 22
train The Three Musketeers 23
train THE PROPOSAL 24
train Leaving Las Vegas by Mike Figgis 25
train "ROCKY" 26
train "SO I MARRIED AN AXE MURDERER" 27
train CHARADE 28
train THE OTHER BOLEYN GIRL 29
train HEAVENLY CREATURES 30
train "The Princess Bride" 31
train "INTOLERABLE CRUELTY" 32
train THE NEXT THREE DAYS 33
train SAVE THE LAST D

In [ ]:
# keys_list_test = []
# for k,v in horror_scripts_raw.items():
#     if k not in keys_list_train:
#         print(k)
#         test_horror.append(v)
#         keys_list_test.append(k)

In [15]:
len(X_train), len(y_train),len(keys_list_train)

(240, 240, 240)

In [16]:
len(X_test), len(y_test), len(keys_list_test)

(60, 60, 60)

In [18]:
X_train[239][0:40]

'GROUNDHOG DAY Written by Danny Rubin Sec'

# Stopwords and Tokenize

In [19]:
stopwords_list = stopwords.words("english")
stopwords_list += string.punctuation
stopwords_list += ["''", '""', '...', '`']
stopwords_list += [str(num) for num in range(0,10)]

In [20]:
def process_script(script):
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    script_tokens_raw =  nltk.regexp_tokenize(script, pattern)
    script_tokens = [word.lower() for word in script_tokens_raw]
    script_words_stopped = [word for word in script_tokens if word not in stopwords_list]
    return script_words_stopped

##### With Lemme

In [82]:
lemmatizer = WordNetLemmatizer()
def process_script_w_lemme(script):
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    script_tokens_raw =  nltk.regexp_tokenize(script, pattern)
    script_tokens = [word.lower() for word in script_tokens_raw]
    script_words_stopped = [lemmatizer.lemmatize(word) for word in script_tokens if word not in stopwords_list]
    return script_words_stopped

In [21]:
processed_script_train = list(map(process_script, X_train))
processed_script_test = list(map(process_script, X_test))

In [84]:
processed_script_train_lemme = list(map(process_script_w_lemme, X_train))
processed_script_test_lemme = list(map(process_script_w_lemme, X_test))

In [22]:
len(processed_script_train), processed_script_train[2][0:3]

(240, ['bonnie', 'clyde', 'david'])

# FreqDist

In [ ]:
# def total_vocab(list_s):
#    total_vocab = set()
#     for list_ in list_s:
#         total_vocab.update(list_)
        

In [23]:
processed_scripts_freqdist_train = list(map(FreqDist, processed_script_train))

In [24]:
processed_scripts_freqdist_train[0]

FreqDist({'zoe': 812, 'stan': 416, 'cont': 127, "i'm": 124, 'int': 99, 'bobby': 98, 'know': 94, 'like': 94, 'mona': 77, 'nana': 77, ...})

##### With Lemme

In [85]:
processed_scripts_freqdist_train_lemme = list(map(FreqDist, processed_script_train_lemme))

In [86]:
processed_scripts_freqdist_train_lemme[0]

FreqDist({'zoe': 812, 'stan': 416, 'look': 133, 'cont': 127, "i'm": 124, 'int': 99, 'bobby': 98, 'know': 97, 'like': 95, 'mona': 77, ...})

# Create Data Frame

In [58]:
train_df = pd.DataFrame(processed_scripts_freqdist_train[0], index=[0])

In [59]:
train_df

,back,plan,written,kate,angelo,august,th,int,exam,room,...,wrote,planted,priest,bjorns,guests,promise,embarrassed,impossible,louis,armstrong's
0,60,5,1,1,1,1,11,99,5,43,...,1,1,2,1,1,4,1,1,1,1


In [60]:
train_df = pd.DataFrame(processed_scripts_freqdist_train[0], index=[0])
for i in range(1,len(processed_scripts_freqdist_train)):
    try:
        right = pd.DataFrame(processed_scripts_freqdist_train[i], index=[i])
        print(right.head(1), right.shape)
        train_df = pd.concat([train_df, right],  ignore_index=True)
        print('success', keys_list_train[i])
    except:
        print('fail', keys_list_train[i])
        continue
        

   pride  prejudice  written  deborah  moggach  int  netherfield  \
1      6          1        1        1        1   81           36   

   hertfordshire  day  vast  ...  exaggerate  occasions  teasing  quarrelling  \
1              9   85     5  ...           1          1        1            1   

   diminish  smaller  star  spangled  fainter  fade  
1         1        2     1         1        2     1  

[1 rows x 3187 columns] (1, 3187)


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


success PRIDE AND PREJUDICE
   bonnie  clyde  david  newman  robert  benton  fade  int  bedroom  close  \
2     543    664      1       1       1       1     6   82       20     48   

   ...  massacre  return  shambles  discern  slumped  level  deputies  result  \
2  ...         1       1         1        1        1      1         1       1   

   tiny  figures  
2     1        1  

[1 rows x 3825 columns] (1, 3825)
success "BONNIE & CLYDE"
   st  elmo  fire  written  joel  schumacher  carl  kurlander  third  draft  \
3  80    69    76        2     1           1     1          1      1      1   

   ...  images  mix  glare  brief  brilliance  krbo  upon  f  mo's  claton  
3  ...       2    1      1      1           1     1     1  1     1       1  

[1 rows x 2935 columns] (1, 2935)
success ST. ELMO'S FIRE
   h  e  r   c  n  p  screenplay  aaron  sorkin  fade  ...  thunderous  \
4  2  5  4  11  5  3           1      1       1     2  ...           1   

   ovation  shouts  bravo  galler

success WILLIAM SHAKESPEARE'S
    three  musketeers  written  david  loughery  revisions  harley  peyton  \
23     46         113        1      1         1          1       1       1   

    material  sole  ...  hugs  valuable  deceive  unused  contortion  twisted  \
23         2     1  ...     1         1        1       1           1        1   

    basks  aloud  helpfully  sparks  
23      1      1          1       1  

[1 rows x 4006 columns] (1, 4006)
success The Three Musketeers
    proposal  written  pete  chiarelli  june  fade  int  margaret  apartment  \
24         2        5     1          1     1     1   63       841          7   

    early  ...  onstage  rug  they've  dances  track  tallest  eagle's  \
24      8  ...        1    1        1       1      1        1        1   

    eaglets  chirp  snuggled  
24        1      1         1  

[1 rows x 2908 columns] (1, 2908)
success THE PROPOSAL
    leaving  las  vegas  mike  figgis
25        1    1      1     1       1 (1, 5)

success A WALK TO REMEMBER
     e  r  n  james  cameron  registered  wgaw  fourth  draft  april  ...  \
44  16  3  1      1        1           1     1       1      1      1  ...   

    mil  trescientos  fifteen  pays  accented  thunderheads  cranes  ribbon  \
44    1            1        1     1         1             1       1       1   

    crescendos  horizon  
44           1        2  

[1 rows x 4366 columns] (1, 4366)
success " T E R M I N A T O R "
    l  e  p  n  jeff  arch  rewritten  nora  ephron  delia  ...  shall  \
45  5  7  3  2     1     1          1     1       2      1  ...      1   

    natural  triumphant  double  fist  gesture  lit  confection  world's  \
45        1           1       1     1        1    1           1        1   

    largest  
45        1  

[1 rows x 2616 columns] (1, 2616)
success S L E E P L E S S   I N   S E A T T L E
    sessions  formerly  surrogate  written  ben  lewin  based  true  story  \
46         4         1          5        1    1  

...

success GINGER SNAPS
     lies  beneath  clark  gregg  june  fade  moving  murky  haze  dark  ...  \
142     3        2      1      1     1     3       6      3     1    12  ...   

     beaded  sweat  builds  crescendo  remain  apartment  glimmering  \
142       1      1       1          1       1          1           1   

     attentively  unsmiling  serene  
142            1          1       1  

[1 rows x 2763 columns] (1, 2763)
success WHAT LIES BENEATH
     grabbers  written  kevin  lehane  production  draft  november  pink  \
143       108        1      1       1         109    109         4     3   

     revised  blue  ...  vista  mental  seem  breakfast  sunrise  ripples  \
143        5     2  ...      1       1     1          1        1        1   

     obvious  tracks  lead  made  
143        1       1     1     1  

[1 rows x 3196 columns] (1, 3196)
success GRABBERS
     wilson  brent  maddock's  tremors
144       1      1          1        1 (1, 4)
success S. S. Wilson 

success "IT'S A WONDERFUL LIFE"
       e  commentary  richard  michaels  titles  extra  terrestial  letters  \
162  360           1        1         1       1      1           1        1   

     soft  purple  ...  earth's  vegetative  memorial  ships  circular  \
162     2       2  ...        1           1         1      1         1   

     rainbow  makeshift  happiness  ends  gazing  
162        1          1          1     1       1  

[1 rows x 2270 columns] (1, 2270)
success "E.T."
     fantastic  four  mark  frost  michael  france  based  marvel  comic  \
163         20    28     1      1        1       1      2       1      1   

     book  ...  operation  listening  cell  leonard's  cellphone  destination  \
163     3  ...          1          1     1          1          1            1   

     stenciled  lettering  freighter  steaming  
163          1          1          1         1  

[1 rows x 3797 columns] (1, 3797)
success FANTASTIC FOUR
     tmnt  written  kevin  munroe  c

success EXCALIBUR
     star  wars  episode  iv  new  hope  journal  whills  george  lucas  ...  \
181   225     4        1   1   14    11        1       1       1      1  ...   

     dignitaries  dress  staggeringly  medallion  winks  repeats  moved  \
181            1      1             1          1      1        1      1   

     assembled  happiness  credits  
181          1          1        1  

[1 rows x 3759 columns] (1, 3759)
success STAR WARS
     h   e  g  r  n  l  screenplay  frank  darabont  novel  ...  immortality  \
182  1  48  7  2  2  1           1      1         1      1  ...            1   

     wished  bedraggled  hungry  match  transforms  nights  fitfully  chasing  \
182       1           1       1      1           1       2         1        1   

     dreams  
182       1  

[1 rows x 3980 columns] (1, 3980)
success T H E   G R E E N   M I L E
     return  jedi  lawrence  kasdan  george  lucas  novel  third  draft  \
183       3    37         1       1       2  

success WATCHMEN
     brazil  screenplay
201       1           1 (1, 2)
success Brazil - the screenplay
     field  dreams  written  phil  alden  robinson  march  final  draft  \
202     91       5        1     1      1         2      1      1      1   

     screenplay  ...  butter  silvering  understands  wisely  hint  moisture  \
202           1  ...       1          1            1       1     1         1   

     forth  higher  bathed  satiny  
202      1       2       1       1  

[1 rows x 3074 columns] (1, 3074)
success ?
     princess  bride  william  goldman  shooting  draft  fade  video  game  \
203        14      6        1        1         2      1     3      2     4   

     computer  ...  rode  arose  swept  pure  invention  rated  snaps  \
203         1  ...     1      1      1     2          1      1      1   

     readying  tipping  hat  
203         1        1    1  

[1 rows x 2822 columns] (1, 2822)
success "The Princess Bride"
     big  fish  written  john  august

success Crouching Tiger, Hidden Dragon
     ted  written  seth  macfarlane  alec  sulkin  wellesley  wild  ext  \
222  594        2     1           1     1       1          1     1   89   

     space  ...  arrested  charged  realized  completely  sounded  trainer  \
222      2  ...         1        1         1           1        1        1   

     substantial  amount  weight  lautner  
222            1       1       1        2  

[1 rows x 3302 columns] (1, 3302)
success TED
     mummy  stephen  sommers  universal  pictures  globe  spins  comes  stop  \
223     18        1        1          1         1      1     22     19    29   

     african  ...  dissolve  plateau  rumble  wraps  snorts  slap  tuk  flap  \
223        1  ...         1        2       1      2       1     1    3     1   

     stolen  sunset  
223       1       1  

[1 rows x 3340 columns] (1, 3340)
success "THE MUMMY"
     batman  screenplay  sam  hamm  first  draft  note  hard  copy  script  \
224     145        

In [ ]:
# df.groupby(df.columns, axis=1).agg(numpy.sum)

In [61]:
train_df.fillna(value=0,axis=0, inplace=True)

In [62]:
train_df.shape

(240, 62253)

In [65]:
train_df.iloc[100:103,10000:10100]

,clock,clock's,clocked,clocker,clocking,clockmaker,clocks,clocktower,clockwise,clockwork,...,club,club's,clubbed,clubbers,clubbing,clubgoers,clubhouse,clubland,clubs,cluck
100,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
full_train_df = train_df.copy()
full_train_df.insert(0, "Movie_name", keys_list_train, False)
full_train_df.insert(1, "Genre_label", y_train, False)

In [69]:
full_train_df.head(10)

,Movie_name,Genre_label,a'baby,a'broad,a'commin,a'couple,a'course,a'cup,a'female,a'hand,...,zzlip,zzr,zzz,zzzing,zzzips,zzzoom,zzzzt,zzzzzz,zzzzzzt,zzzzzzzzzzzzzz
0,THE BACK-UP PLAN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PRIDE AND PREJUDICE,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"""BONNIE & CLYDE""",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ST. ELMO'S FIRE,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,T H E A M E R I C A N P R E S I D E N T,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,LOG #028,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,MONTE CARLO,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,PUNCH-DRUNK LOVE by Paul Thomas Anderson,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"""CRASH""",0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,MRS BROWN BY JEREMY BROCK,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### With Lemme

In [87]:
train_df_lemme = pd.DataFrame(processed_scripts_freqdist_train_lemme[0], index=[0])
for i in range(1,len(processed_scripts_freqdist_train_lemme)):
    try:
        right_lemme = pd.DataFrame(processed_scripts_freqdist_train_lemme[i], index=[i])
        print(right_lemme.head(1), right_lemme.shape)
        train_df_lemme = pd.concat([train_df_lemme, right_lemme],  ignore_index=True)
        print('success', keys_list_train[i])
    except:
        print('fail', keys_list_train[i])
        continue

   pride  prejudice  written  deborah  moggach  int  netherfield  \
1      6          1        1        1        1   81           36   

   hertfordshire  day  vast  ...  liveliness  virtue  exaggerate  teasing  \
1              9   89     5  ...           1       1           1        1   

   quarrelling  diminish  smaller  star  spangled  fainter  
1            1         1        2     1         1        2  

[1 rows x 2922 columns] (1, 2922)


/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


success PRIDE AND PREJUDICE
   bonnie  clyde  david  newman  robert  benton  fade  int  bedroom  close  \
2     543    664      1       1       1       1     6   82       20     49   

   ...  careful  teaching  pack  massacre  shamble  discern  slumped  level  \
2  ...        1         1     1         1        1        1        1      1   

   result  tiny  
2       1     1  

[1 rows x 3499 columns] (1, 3499)
success "BONNIE & CLYDE"
   st  elmo  fire  written  joel  schumacher  carl  kurlander  third  draft  \
3  80    69    76        2     2           1     1          1      1      1   

   ...  flare  image  mix  brief  brilliance  krbo  upon  f  mo's  claton  
3  ...      1      2    1      1           1     1     1  1     1       1  

[1 rows x 2689 columns] (1, 2689)
success ST. ELMO'S FIRE
   h  e  r   c  n  p  screenplay  aaron  sorkin  fade  ...  doorkeeper  leap  \
4  2  5  4  11  5  3           1      1       1     3  ...           1     1   

   thunderous  ovation  bravo

success WILLIAM SHAKESPEARE'S
    three  musketeer  written  david  loughery  revision  harley  peyton  \
23     46        150        1      1         1         1       1       1   

    material  sole  ...  hug  valuable  deceive  unused  contortion  twisted  \
23         2     1  ...    1         1        1       1           1        1   

    basks  aloud  helpfully  spark  
23      1      1          1      1  

[1 rows x 3596 columns] (1, 3596)
success The Three Musketeers
    proposal  written  pete  chiarelli  june  fade  int  margaret  apartment  \
24         2        5     1          1     1     1   63       841          7   

    early  ...  slide  onstage  rug  they've  track  tallest  eagle's  eaglet  \
24      8  ...      1        1    1        1      1        1        1       1   

    chirp  snuggled  
24      1         1  

[1 rows x 2642 columns] (1, 2642)
success THE PROPOSAL
    leaving  la  vega  mike  figgis
25        1   1     1     1       1 (1, 5)
success Leaving

success A WALK TO REMEMBER
     e  r  n  james  cameron  registered  wgaw  fourth  draft  april  ...  \
44  16  3  1      1        1           1     1       1      1      1  ...   

    snapshot  ruffling  mil  trescientos  fifteen  accented  thunderhead  \
44         1         1    1            1        1         1            1   

    ribbon  crescendo  horizon  
44       1          1        2  

[1 rows x 3884 columns] (1, 3884)
success " T E R M I N A T O R "
    l  e  p  n  jeff  arch  rewritten  nora  ephron  delia  ...  throat  \
45  5  7  3  2     1     1          1     1       2      1  ...       1   

    shall  natural  triumphant  double  fist  lit  confection  world's  \
45      1        1           1       1     1    1           1        1   

    largest  
45        1  

[1 rows x 2405 columns] (1, 2405)
success S L E E P L E S S   I N   S E A T T L E
    session  formerly  surrogate  written  ben  lewin  based  true  story  \
46       10         1          7        1   

...

success GRABBERS
     wilson  brent  maddock's  tremor
144       1      1          1       1 (1, 4)
success S. S. Wilson & Brent Maddock's "Tremors"
     alien  resurrection  ext  deep  space  silent  black  sweep  slowly  \
145    136             1   11     4     16       7     12      3      25   

     across  ...  love  answering  official  grateful  suggestion  chrissake  \
145      12  ...     1          1         1         1           1          1   

     plenty  stranger  dwindles  dancing  
145       1         1         1        1  

[1 rows x 2903 columns] (1, 2903)
success Alien Resurrection
     title  sequence  credit  run  intercut  following  two  ext  nevada  \
146      2         3       3   10         4          1   19   54       5   

     desert  ...  cacophony  rim  peep  horizon  settle  reign  virtually  \
146      13  ...          1    1     1        1       1      1          1   

     destroyed  leaving  plume  
146          1        1      1  

[1 rows x 2666

success FANTASTIC FOUR
     tmnt  written  kevin  munroe  created  eastman  peter  laird  first  \
164     9        1      2       1        1        1      2      1     14   

     draft  ...  taped  waist  horsing  geared  hoot  holler  bathroom  \
164      1  ...      1      1        1       1     1       1         1   

     provided  loud  easy  
164         1     1     1  

[1 rows x 3338 columns] (1, 3338)
success TMNT
     pirate  carribean  dead  man  chest  written  ted  elliott  terry  \
165      14          1     7   38     52        1    1        1      1   

     rossio  ...  world's  dose  boot  descends  barbossa  apple  juice  \
165       1  ...        1     1     1         1         2      1      1   

     dripples  chin  barbossa's  
165         1     1           1  

[1 rows x 2451 columns] (1, 2451)
success PIRATES OF THE CARRIBEAN: DEAD MAN'S CHEST
     kate  leopold  screenplay  james  mangold  steven  rogers  based  story  \
166   590      690           1      1

success T H E   G R E E N   M I L E
     return  jedi  lawrence  kasdan  george  lucas  novel  third  draft  \
183       7    37         1       1       2      2      1      1      1   

     property  ...  hugged  greet  hardy  comfort  other's  original  \
183         1  ...       1      1      1        1        1         1   

     adventurer  sideline  elsewhere  fade  
183           1         1          1     1  

[1 rows x 3003 columns] (1, 3003)
success RETURN
     lost  horizon  written  robert  previous  hit  riskin  based  novel  \
184     8        4        2       7         4    7       1      2      1   

     james  ...  toast  sincerity  sparkling  pelting  ascended  steep  gait  \
184      1  ...      1          1          1        1         1      1     1   

     incline  silhouette  ring  
184        1           1     1  

[1 rows x 3059 columns] (1, 3059)
success LOST HORIZON
     ext  space  vast  sea  star  severas  backdrop  main  title  following  \
185   79     

success "The Princess Bride"
     big  fish  written  john  august  based  novel  daniel  wallace  final  \
204   26    29        4     1       1      1      1       1        1      6   

     ...  huge  exists  graveside  grave  stunning  conjoined  forgotten  \
204  ...     1       1          1      1         1          1          1   

     monument  screen  lady's  
204         1       1       1  

[1 rows x 3171 columns] (1, 3171)
success "BIG FISH"
     batman  screenplay  sam  hamm  based  character  created  bob  kane  \
205     169           1    1     1      1          5        2    3     2   

     first  ...  sparkle  settling  brief  skyward  searing  dominating  \
205     16  ...        1         1      1        1        1           1   

     glaring  vanish  plunging  adventure  
205        1       1         1          1  

[1 rows x 4564 columns] (1, 4564)
success BATMAN
     twilight  new  moon  written  melissa  rosenberg  adapted  novel  \
206         6    7     5  

success "THE MUMMY"
     batman  screenplay  sam  hamm  first  draft  note  hard  copy  script  \
224     145           1    1     1     16      1     6    15     8       2   

     ...  cleaning  cd  dunno  plug  produce  devastated  tossing  furniture  \
224  ...         1   1      1     1        1           1        1          1   

     image  decidedly  
224      1          1  

[1 rows x 4859 columns] (1, 4859)
success BATMAN 2
     fifth  element  original  script  luc  besson  revision  robert  mark  \
225      8       23         1       1    2       2         1       1     1   

     kamen  ...  storm  worthless  account  plotz  ignored  sacrificed  pan  \
225      1  ...      1          1        1      1        1           1    1   

     rising  silver  ascend  
225       1       1       1  

[1 rows x 3176 columns] (1, 3176)
success The Fifth Element
     wonder  woman  screenplay  allan  heinberg  story  zack  snyder  jason  \
226      11     41           1      2         

In [91]:
train_df_lemme.shape

(240, 55217)

In [88]:
train_df_lemme.fillna(value=0,axis=0, inplace=True)

In [89]:
full_train_df_lemme = train_df_lemme.copy()
full_train_df_lemme.insert(0, "Movie_name", keys_list_train, False)
full_train_df_lemme.insert(1, "Genre_label", y_train, False)

In [90]:
full_train_df_lemme.shape

(240, 55219)

# TF-IDF

In [70]:
vectorizer = TfidfVectorizer()

In [72]:
tf_idf_data_train = vectorizer.fit_transform(train_df)

In [73]:
tf_idf_data_train

<62253x57641 sparse matrix of type '<class 'numpy.float64'>'
	with 62758 stored elements in Compressed Sparse Row format>

In [ ]:
tf_idf_data_test = vectorizer.transform()